### Inicjalizacja środowiska

In [1]:
!pip install textblob
!python -m textblob.download_corpora lite

[nltk_data] Downloading package brown to /home/mc/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/mc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Finished.


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.param import Param
from pyspark.ml.param.shared import HasInputCol, HasOutputCol

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.types import * 
from pyspark.ml.linalg import Vectors, VectorUDT

from pyspark.sql.functions import floor, rand, udf, max
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import json

from post_extractor.posts import (
    PostTransformer,
    TranslateTransformer,
    BasicSpeechPartsTransformer
)

sconf = SparkConf()              \
    .setMaster('local[*]')       \
    .setAppName('PipelineFlow')

sc = SparkContext.getOrCreate(sconf)
sess = SparkSession(sc)
sqlContext = SQLContext(sc)
    

### Wczytywanie plików

In [3]:
def load_posts(spark_ctx, files):
    rdd = spark_ctx.wholeTextFiles(files)
    rdd = rdd.map(lambda x: (x[0], json.loads(x[1])))
    df = rdd.toDF(['file', 'content'], sampleRatio=0.2)
    return df

### Przykład zastosowania TransformerProxy do automatyzacji ewaluacji

`TransformerProxy` jest obiektem który opakowuje inny Transformer. Dzięki temu, możliwe jest stworzenie uniwersalnego pipeline'u (jak na rysunku poniżej) bez specyfikowania od razu konkretnych implementacji poszczególnych etapów. Np. wiemy, że pierwszy etap parsuje plik html dzieląc go na słowa, drugi etap usuwa obrazki a trzeci rozpoznaje i zlicza czasowniki, ale nie wiemy jakie konkretne implementacje będziemy chcieli dostarczyć dla poszczególnych etapów.
 
W szczególności jeśli bedziemy chcieli mieć wiele różnych implementacji dla tego samego etapu opisane podejście będzie użyteczne. Testowanie takiego przepływu będzie odbywało się za pomocą klasy CrossValidator która zostanie opisana później. Na razie wspomnijmy jedynie o tym, że CrossValidator nie potrafi modyfikować pipeline'u poprzez zamianę np. jednego transformera na drugi, potrafi natomiast modyfikować parametry kolejnych etapów przepływu. Dzięki zastosowanemu podejściu CrossValidator będzie w stanie testować kombinację różnych implementacji poszególnych etapów pipeline'u.

![title](pipeline.jpg)

### Klasa TransformerProxy

In [4]:
class DenseVectorTransformer(Transformer, HasInputCol, HasOutputCol):
    def __init__(self):
        super(DenseVectorTransformer, self).__init__()
    def _transform(self, dataset):
        toDenseVector = udf(lambda arr: Vectors.dense(arr), VectorUDT())
        return dataset.withColumn(self.getOutputCol(), toDenseVector(self.getInputCol()))

In [5]:
class TransformerProxy(Transformer):

    def __init__(self):
        super(TransformerProxy, self).__init__()
        self.transformer = Param(self, "transformer", "")

    def set_transformer(self, transformer):
        self._paramMap[self.transformer] = transformer
        return self

    def get_transformer(self):
        return self.getOrDefault(self.transformer)

    def _transform(self, dataset):
        return self.get_transformer().transform(dataset)

### Utworzenie instancji transformerów

W przygotowanym pipeline możemy wykorzystać w pierwszym TransformerProxy trzy różne implementacje: 
- MeanFeaturesTransformer 
- MedianFeaturesTransformer 
- NumerOfOccurrencesFeaturesTransformer 

Możemy je zatem przekazać do abstrakcji ParamGridBuilder'a, który będzie parametrem przekazanym do klasy CrossValidator.   

In [6]:
import pprint
pprint.pprint(TranslateTransformer.__doc__)

('\n'
 '    Klasa TranslateTransformer dziedziczy po klasach pyspark.ml.Transformer, '
 'pyspark.ml.param.shared.HasInputCol,\n'
 '    pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która '
 'przyjmuje na wejściu obiekt typu dataframe.\n'
 '    Metoda ta tłumaczy tekst zawarty w kolumnie inputCol  z języka polskiego '
 'na angielski i umieszcza go w kolumnie\n'
 '    outputCol.\n'
 '    ')


In [7]:
pprint.pprint(BasicSpeechPartsTransformer.__doc__)

('\n'
 '    Klasa BasicSpeechPartsTransformer dziedziczy  po klasach '
 'pyspark.ml.Transformer, pyspark.ml.param.shared.HasInputCol,\n'
 '    pyspark.ml.param.shared.HasOutputCol. Posiada metodę transform, która '
 'przyjmuje na wejściu obiekt typu dataframe.\n'
 '    Metoda ta z tekstu zawartego w kolumnie inputCol zlicza wystąpienie '
 'podstawowych części mowy (rzeczownik, czasownik, przymiotnik)\n'
 '    i wstawia do outputCol w postaci tablicy wartości.\n'
 '    ')


In [8]:
poster = PostTransformer()
poster.setInputCol('content').setOutputCol('posts')

translator = TranslateTransformer()
translator.setInputCol('posts').setOutputCol('translated')

speech_parter = BasicSpeechPartsTransformer()
speech_parter.setInputCol('translated').setOutputCol('speech_parts')

dv_transformer = DenseVectorTransformer()
dv_transformer.setInputCol('speech_parts').setOutputCol('features')

dt = DecisionTreeClassifier(labelCol='label')

### Przygotowanie modyfikowalnego pipeline'u
W tej wersji, wszystkie istniejące wczesniej stage zastępujemy obiektami `TransformerProxy`

In [9]:
parameterized_pipeline = Pipeline(stages=[
    poster,
    translator,
    speech_parter,
    dv_transformer,
    dt
])

In [10]:
data = load_posts(sc, "data/posts/*")
dataWithLabels = data.withColumn('label', floor(rand() * 3).cast(DoubleType()))

CrossValidator uwzględniając wszystkie kombinacje dostarczonych parametrów wskazuje który zestaw parametrów cechuje się najlepszymi wynikami.

In [11]:
# CrossValidator will automatically find the best set of parameters
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='label')

cv_results = {}

paramGrid = ParamGridBuilder() \
  .addGrid(dt.maxBins, [5, 10, 15, 20]) \
  .build()

cv = CrossValidator(estimator=parameterized_pipeline,
                   estimatorParamMaps=paramGrid,
                   evaluator=evaluator,
                   numFolds=3)
cv_result = cv.fit(dataWithLabels)

### Wypisanie nazw transformerow wybranych przez CV

In [12]:
prediction = cv_result.transform(data)
selected = prediction.select("file", "speech_parts", "probability", "prediction")

In [14]:
for row in selected.collect():
    print(row)

Row(file='file:/home/mc/Projects/ZTNBD-ZAD/data/posts/kascysko.blogspot.com.146.json', speech_parts=[79, 54, 31], probability=DenseVector([1.0, 0.0]), prediction=0.0)
Row(file='file:/home/mc/Projects/ZTNBD-ZAD/data/posts/kascysko.blogspot.com.142.json', speech_parts=[94, 83, 33], probability=DenseVector([0.0, 1.0]), prediction=1.0)
Row(file='file:/home/mc/Projects/ZTNBD-ZAD/data/posts/kascysko.blogspot.com.118.json', speech_parts=[48, 43, 17], probability=DenseVector([0.0, 1.0]), prediction=1.0)
